In [1]:
import sys
sys.path.append("..")
from nba_py import player, game
import pandas as pd
import numpy as np
import json
import time
import random
import requests
import time

from IPython.display import display

pd.options.display.max_columns = None

In [2]:
def getPlayerGames(pid):
    response = player.PlayerGameLogs(pid,season='2017-18')

    rjson = response.json['resultSets'][0]
    headers= rjson['headers']
    rowSet = rjson['rowSet']
    rows = rowSet[1:]


    gameLog = pd.DataFrame(rows, columns=headers)
    gameIDs = gameLog.Game_ID
    return gameIDs
lbjGames = getPlayerGames(2544)

In [3]:
def getGamePBP(gid):
    url = 'http://stats.nba.com/stats/playbyplayv2/?GameID={}&EndPeriod=0&StartPeriod=0'.format(gid)
    headers = {'host': 'stats.nba.com',
              'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
              'cache-control': 'no-cache',
              'pragma': 'no-cache',
              'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
               'accept-encoding': 'gzip, deflate',
            'accept-language': 'en-US,en;q=0.9',
            'connection': 'keep-alive',
              'upgrade-insecure-requests': '1'}
    
    r = requests.get(url,  headers=headers)
    jsn = r.json()
    rjson = jsn['resultSets'][0]
    headers= rjson['headers']
    rowSet = rjson['rowSet']
    rows = rowSet[1:]

    df = pd.DataFrame(rows, columns=headers)
    return df
# getGamePBP(lbjGames[0])

In [4]:
def getPlayerPBP(games):
    i =1
    df = getGamePBP(games[0])
    for g in games[1:]:
        df = df.append(getGamePBP(g))
        if i % 100 == 0:
            print(i)
            df.to_csv('pbp-2017-18.csv')
        time.sleep(random.randint(1, 3))
        i = i + 1
    return df
# lbjDF = getPlayerPBP(lbjGames)

In [5]:
lbjDF.head(100)
pbp = lbjDF

NameError: name 'lbjDF' is not defined

In [ ]:
pbp[cavs.EVENTMSGTYPE == 5].EVENTMSGACTIONTYPE.unique()

In [ ]:
pbp[(cavs.EVENTMSGTYPE == 5)&(cavs.EVENTMSGACTIONTYPE == 45)]

In [ ]:
pbp.head(150)

In [ ]:
len(pbp[pbp.TO_PID == 2544])

In [ ]:
gameIDs = [str(x).zfill(10) for x in range(21700001,21701230)]
gameIDs

In [ ]:
# pbpdf = getPlayerPBP(gameIDs)

In [ ]:
pbpdf.to_csv('pbp-2017-18.csv')

In [ ]:
pbpdf.tail()

In [ ]:
pbp = pbpdf.append(getGamePBP('0021701230'))

In [ ]:
pbp.to_csv('pbp-2017-18.csv')

In [ ]:
pbp['MISS_3PT_PID'] = np.where(((pbp.EVENTMSGTYPE == 2)&(pbp.HOMEDESCRIPTION.str.contains('3PT'))) |
                                ((pbp.EVENTMSGTYPE == 2)&(pbp.VISITORDESCRIPTION.str.contains('3PT')))
                                 ,
                                pbp.PLAYER1_ID,
                                None)
pbp['MAKE_3PT_PID'] = np.where(((pbp.EVENTMSGTYPE == 1)&(pbp.HOMEDESCRIPTION.str.contains('3PT'))) |
                                ((pbp.EVENTMSGTYPE == 1)&(pbp.VISITORDESCRIPTION.str.contains('3PT')))
                                 ,
                                pbp.PLAYER1_ID,
                                None)
pbp['MISS_2PT_PID'] = np.where(((pbp.EVENTMSGTYPE == 2)& (pbp.HOMEDESCRIPTION.str.contains('3PT')==False)) |
                                ((pbp.EVENTMSGTYPE == 2)&(pbp.VISITORDESCRIPTION.str.contains('3PT')==False))
                                 ,
                                pbp.PLAYER1_ID,
                                None)
pbp['MAKE_2PT_PID'] = np.where(((pbp.EVENTMSGTYPE == 1)&(pbp.HOMEDESCRIPTION.str.contains('3PT')==False)) |
                                ((pbp.EVENTMSGTYPE == 1)&(pbp.VISITORDESCRIPTION.str.contains('3PT')==False))
                                 ,
                                pbp.PLAYER1_ID,
                                None)
pbp['MISS_FT_PID'] = np.where(((pbp.EVENTMSGTYPE == 3)& (pbp.HOMEDESCRIPTION.str.contains('MISS'))) |
                                ((pbp.EVENTMSGTYPE == 3)&(pbp.VISITORDESCRIPTION.str.contains('MISS')))
                                 ,
                                pbp.PLAYER1_ID,
                                None)
pbp['MAKE_FT_PID'] = np.where(((pbp.EVENTMSGTYPE == 3)&(pbp.HOMEDESCRIPTION.str.contains('MISS')==False)) |
                                ((pbp.EVENTMSGTYPE == 3)&(pbp.VISITORDESCRIPTION.str.contains('MISS')==False))
                                 ,
                                pbp.PLAYER1_ID,
                                None)
pbp['AST_BY_PID'] = np.where(((pbp.EVENTMSGTYPE == 1)&(pbp.HOMEDESCRIPTION.str.contains('AST'))) |
                                ((pbp.EVENTMSGTYPE == 1)&(pbp.VISITORDESCRIPTION.str.contains('AST')))
                                 ,
                                pbp.PLAYER2_ID,
                                None)
pbp['AST_TO_PID'] = np.where(((pbp.EVENTMSGTYPE == 1)&(pbp.HOMEDESCRIPTION.str.contains('AST'))) |
                                ((pbp.EVENTMSGTYPE == 1)&(pbp.VISITORDESCRIPTION.str.contains('AST')))
                                 ,
                                pbp.PLAYER1_ID,
                                None)
pbp['TO_PID'] = np.where((pbp.EVENTMSGTYPE == 5),
                                pbp.PLAYER1_ID,
                                None)
pbp['STL_PID'] = np.where((pbp.EVENTMSGTYPE == 5) &((pbp.EVENTMSGACTIONTYPE == 1) |(pbp.EVENTMSGACTIONTYPE == 2))
                                 ,
                                pbp.PLAYER2_ID,
                                None)
pbp['BLK_PID']  = np.where(((pbp.EVENTMSGTYPE == 2)&(pbp.HOMEDESCRIPTION.str.contains('BLOCK'))) |
                                ((pbp.EVENTMSGTYPE == 1)&(pbp.VISITORDESCRIPTION.str.contains('BLOCK')))
                                 ,
                                pbp.PLAYER3_ID,
                                None)
pbp['PF_PID'] = np.where((pbp.EVENTMSGTYPE == 6),
                                pbp.PLAYER1_ID,
                                None)

In [ ]:
def parseTurnovers(row):
    if ((row['EVENTMSGTYPE'] == 5) & (row['EVENTMSGACTIONTYPE'] == 1)):
        return 'bad-pass-live'
    elif ((row['EVENTMSGTYPE'] == 5) & (row['EVENTMSGACTIONTYPE'] == 45)):
        return 'bad-pass-dead'
    elif ((row['EVENTMSGTYPE'] == 5) & (row['EVENTMSGACTIONTYPE'] == 2)):
        return 'lost-ball-live'
    elif ((row['EVENTMSGTYPE'] == 5) & (row['EVENTMSGACTIONTYPE'] == 40)):
        return 'lost-ball-dead'
    elif ((row['EVENTMSGTYPE'] == 5) & (row['EVENTMSGACTIONTYPE'] == 39)):
        return 'lost-ball-dead'
    elif ((row['EVENTMSGTYPE'] == 5) & (row['EVENTMSGACTIONTYPE'] == 37)):
        return 'off-foul'
    else:
        if (row['EVENTMSGTYPE'] == 5):
            return 'violation'
        else:
            return None

pbp['TO_TYPE'] = pbp.apply(parseTurnovers, axis=1)

In [ ]:
pbp['SHIFTED_PERSON1_TYPE'] = pbp.PERSON1TYPE.shift()
pbp['OREB_PID'] = np.where((pbp.EVENTMSGTYPE == 4)&(pbp.PERSON1TYPE == pbp.SHIFTED_PERSON1_TYPE) 
                                 ,
                                pbp.PLAYER1_ID,
                                None)
pbp['DREB_PID'] = np.where((pbp.EVENTMSGTYPE == 4)&(pbp.PERSON1TYPE != pbp.SHIFTED_PERSON1_TYPE) 
                                 ,
                                pbp.PLAYER1_ID,
                                None)

In [ ]:
pbp.head(100)

In [ ]:
pbp.to_csv('pbp-2017-18.csv')

In [ ]:
len(pbp[pbp.TO_PID == 2544])

In [ ]:
pbp[(pbp.EVENTMSGTYPE == 4) ]

In [60]:
pbp = pd.read_csv('docs/pbp-2017-18.csv')

In [41]:
pbp.head()

Unnamed: 0   GAME_ID  EVENTNUM  EVENTMSGTYPE  EVENTMSGACTIONTYPE  PERIOD  \
0           0  21700001         4            10                   0       1   
1           1  21700001         7             1                 101       1   
2           2  21700001         9             2                   5       1   
3           3  21700001        11             4                   0       1   
4           4  21700001        12             2                   1       1   

  WCTIMESTRING PCTIMESTRING                            HOMEDESCRIPTION  \
0      8:04 PM        12:00  Jump Ball Love vs. Horford: Tip to Irving   
1      8:04 PM        11:44                                        NaN   
2      8:04 PM        11:27                         MISS Rose 2' Layup   
3      8:05 PM        11:23                                        NaN   
4      8:05 PM        11:21                                        NaN   

   NEUTRALDESCRIPTION                                 VISITORDESCRIPTION  \
0                 NaN                                                NaN   
1                 NaN  Irving 10' Driving Floating Jump Shot (2 PTS) ...   
2                 NaN                              Horford BLOCK (1 BLK)   
3                 NaN                      Horford REBOUND (Off:0 Def:1)   
4                 NaN                     MISS Hayward 25' 3PT Jump Shot   

   SCORE SCOREMARGIN  PERSON1TYPE  PLAYER1_ID    PLAYER1_NAME  \
0    NaN         NaN          4.0      201567      Kevin Love   
1  2 - 0          -2          5.0      202681    Kyrie Irving   
2    NaN         NaN          4.0      201565    Derrick Rose   
3    NaN         NaN          5.0      201143      Al Horford   
4    NaN         NaN          5.0      202330  Gordon Hayward   

   PLAYER1_TEAM_ID PLAYER1_TEAM_CITY PLAYER1_TEAM_NICKNAME  \
0     1.610613e+09         Cleveland             Cavaliers   
1     1.610613e+09            Boston               Celtics   
2     1.610613e+09         Cleveland             Cavaliers   
3     1.610613e+09            Boston               Celtics   
4     1.610613e+09            Boston               Celtics   

  PLAYER1_TEAM_ABBREVIATION  PERSON2TYPE  PLAYER2_ID PLAYER2_NAME  \
0                       CLE            5      201143   Al Horford   
1                       BOS            5      201143   Al Horford   
2                       CLE            0           0          NaN   
3                       BOS            0           0          NaN   
4                       BOS            0           0          NaN   

   PLAYER2_TEAM_ID PLAYER2_TEAM_CITY PLAYER2_TEAM_NICKNAME  \
0     1.610613e+09            Boston               Celtics   
1     1.610613e+09            Boston               Celtics   
2              NaN               NaN                   NaN   
3              NaN               NaN                   NaN   
4              NaN               NaN                   NaN   

  PLAYER2_TEAM_ABBREVIATION  PERSON3TYPE  PLAYER3_ID  PLAYER3_NAME  \
0                       BOS            5      202681  Kyrie Irving   
1                       BOS            0           0           NaN   
2                       NaN            5      201143    Al Horford   
3                       NaN            0           0           NaN   
4                       NaN            0           0           NaN   

   PLAYER3_TEAM_ID PLAYER3_TEAM_CITY PLAYER3_TEAM_NICKNAME  \
0     1.610613e+09            Boston               Celtics   
1              NaN               NaN                   NaN   
2     1.610613e+09            Boston               Celtics   
3              NaN               NaN                   NaN   
4              NaN               NaN                   NaN   

  PLAYER3_TEAM_ABBREVIATION  MISS_3PT_PID  MAKE_3PT_PID  MISS_2PT_PID  \
0                       BOS           NaN           NaN           NaN   
1                       NaN           NaN           NaN           NaN   
2                       BOS           NaN           NaN      

In [61]:
pbp['PF_PID'] = np.where((pbp.EVENTMSGTYPE == 6),
                                pbp.PLAYER1_ID,
                                None)

In [43]:
pbp.head(100)

Unnamed: 0   GAME_ID  EVENTNUM  EVENTMSGTYPE  EVENTMSGACTIONTYPE  PERIOD  \
0            0  21700001         4            10                   0       1   
1            1  21700001         7             1                 101       1   
2            2  21700001         9             2                   5       1   
3            3  21700001        11             4                   0       1   
4            4  21700001        12             2                   1       1   
5            5  21700001        13             4                   0       1   
6            6  21700001        14             2                   1       1   
7            7  21700001        15             4                   0       1   
8            8  21700001        16             2                  98       1   
9            9  21700001        18             4                   0       1   
10          10  21700001        20             1                   1       1   
11          11  21700001        22             6                  17       1   
12          12  21700001        23             3                  16       1   
13          13  21700001        24             1                   1       1   
14          14  21700001        26             5                  21       1   
15          15  21700001        27             6                   1       1   
16          16  21700001        29             1                  86       1   
17          17  21700001        31             2                   9       1   
18          18  21700001        32             4                   0       1   
19          19  21700001        33             1                  41       1   
20          20  21700001        35             6                   1       1   
21          21  21700001        37             1                  63       1   
22          22  21700001        39             2                   1       1   
23          23  21700001        40             4                   0       1   
24          24  21700001        41             6                   2       1   
25          25  21700001        43             3                  11       1   
26          26  21700001        44             3                  12       1   
27          27  21700001        45             2                   6       1   
28          28  21700001        46             4                   0       1   
29          29  21700001        48             2                  63       1   
..         ...       ...       ...           ...                 ...     ...   
70          70  21700001       102             8                   0       1   
71          71  21700001       103             3                  10       1   
72          72  21700001       104             1                  43       1   
73          73  21700001       106             2                   1       1   
74          74  21700001       107             4                   0       1   
75          75  21700001       108             2                   1       1   
76          76  21700001       109             4                   0       1   
77          77  21700001       110             2                   6       1   
78          78  21700001       111             4                   0       1   
79          79  21700001       112             1                   1       1   
80          80  21700001       114             2                   1       1   
81          81  21700001       115             4                   0       1   
82          82  21700001       116             5                   2       1   
83          83  21700001       118             1                  41       1   
84          84  21700001       679             6                   2       1   
85          85  21700001       120             9                   1       1   
86          86  21700001       128             8                   0       1   
87          87  21700001       129             8                   0       1   
88  

In [62]:
pbp2 = pbp.drop(columns=['Unnamed: 0','EVENTMSGTYPE','EVENTMSGACTIONTYPE',
                  'WCTIMESTRING','PCTIMESTRING','NEUTRALDESCRIPTION', 'HOMEDESCRIPTION',
                 'VISITORDESCRIPTION'])

In [63]:
pbp2.to_csv('short-pbp-2017-18.csv')

In [22]:
unID = pbp.PLAYER1_ID.unique()
unName = pbp.PLAYER1_NAME.unique()
unID = unID[~np.isnan(unID)]
print(len(unID))
# unName = unName[~np.isna(unName)]
print(len(unName))
# playerDF = pd.DataFrame({'pid':unID,'pName':unName})
# playerDF.head()

1252
823


In [45]:
dff = pbp.dropna(subset=['HOMEDESCRIPTION','NEUTRALDESCRIPTION', 'VISITORDESCRIPTION'], how='all')

In [54]:
playerDF = dff[['PLAYER1_ID','PLAYER1_NAME']].dropna()
playerDF = playerDF.drop_duplicates()
temp= playerDF.PLAYER1_NAME.str.split()
playerDF['LastName'] = temp.str[-1]
playerDF.sort_values(by='LastName',inplace=True)
playerDF.head()

PLAYER1_ID   PLAYER1_NAME LastName
6645       203518   Alex Abrines  Abrines
1430       203112     Quincy Acy      Acy
6593       203500   Steven Adams    Adams
2016      1628389    Bam Adebayo  Adebayo
10085      201167  Arron Afflalo  Afflalo

In [55]:
playerDF.to_csv('player-data.csv')

Unnamed: 0   GAME_ID  EVENTNUM  EVENTMSGTYPE  EVENTMSGACTIONTYPE  PERIOD  \
0             0  21700001         4            10                   0       1   
1             1  21700001         7             1                 101       1   
2             2  21700001         9             2                   5       1   
3             3  21700001        11             4                   0       1   
4             4  21700001        12             2                   1       1   
5             5  21700001        13             4                   0       1   
6             6  21700001        14             2                   1       1   
7             7  21700001        15             4                   0       1   
8             8  21700001        16             2                  98       1   
9             9  21700001        18             4                   0       1   
10           10  21700001        20             1                   1       1   
11           11  21700001        22             6                  17       1   
12           12  21700001        23             3                  16       1   
13           13  21700001        24             1                   1       1   
14           14  21700001        26             5                  21       1   
15           15  21700001        27             6                   1       1   
16           16  21700001        29             1                  86       1   
17           17  21700001        31             2                   9       1   
18           18  21700001        32             4                   0       1   
19           19  21700001        33             1                  41       1   
20           20  21700001        35             6                   1       1   
21           21  21700001        37             1                  63       1   
22           22  21700001        39             2                   1       1   
23           23  21700001        40             4                   0       1   
24           24  21700001        41             6                   2       1   
25           25  21700001        43             3                  11       1   
26           26  21700001        44             3                  12       1   
27           27  21700001        45             2                   6       1   
28           28  21700001        46             4                   0       1   
29           29  21700001        48             2                  63       1   
..          ...       ...       ...           ...                 ...     ...   
190         190  21700001       278             2                  47       2   
191         191  21700001       279             4                   0       2   
192         192  21700001       280             2                   1       2   
193         193  21700001       281             4                   0       2   
194         194  21700001       282             1                 101       2   
195         195  21700001       283             2                   1       2   
196         196  21700001       284             4                   0       2   
197         197  21700001       285             1                  44       2   
198         198  21700001       287             6                   1       2   
199         199  21700001       291             8                   0       2   
200         200  21700001       292             8                   0       2   
201         201  21700001       293             1                  80       2   
202         202  21700001       294             2                   1       2   
203         203  21700001       295             4                   0       2   
204         204  21700001       296             2                   6       2   
205         205  21700001       297             4                   0       2   
206         206  21700001       298             2                   5       2   
207         207  21700001       300 